In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from gensim.models.ldamodel import LdaModel
from ObjectivelyFunny.LDA_trainer import LDATrainer
from ObjectivelyFunny.pipeline import set_pipeline
from ObjectivelyFunny.data import get_data, sent_to_words
from ObjectivelyFunny import word_selections

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (LDA_trainer.py, line 70)

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
lda = LdaModel.load('../model')

In [ ]:
lda

In [ ]:
id2word = Dictionary.load('../model.id2word')

In [ ]:
df = get_data(gender=[1])
print('Data acquired')
clean_steps = ['music', 'brackets', 'lowercase', 'regex', 'numbers', 'uncensor', 'remove', 'punctuation',
'lemmatizer', 'manual_lemmatize', 'remove2']
clean_df = set_pipeline(clean_steps,
                dropword_list = word_selections.standard_dropword_list + word_selections.decade_dropword_list
                ).fit_transform(df)
print('Dataframe cleaned')

model = LDATrainer()
model.make_words(clean_df).make_grams().make_dictionary()
print('Gram dictionary made')
model.run()
print('complete')

In [ ]:
lda.print_topics()

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, model.corpus, model.id2word, mds='mmds')
vis

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
for t in range(lda.num_topics):
    plt.figure()
    plt.imshow(WordCloud().fit_words(dict(lda.show_topic(t, 200))))
    plt.axis("off")
    plt.title("Topic #" + str(t))
    plt.show()